In [22]:
#import the shit

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
import tensorflow as tf
from nltk.stem import WordNetLemmatizer 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import confusion_matrix, classification_report
import warnings
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

warnings.filterwarnings('ignore')

In [23]:
data = pd.read_csv('dataset.csv')
data = data.dropna()
X = data['selected_text']
y = data['sentiment']
print(y.value_counts())

sentiment
0    11104
2     8572
1     7671
Name: count, dtype: int64


In [24]:
#define a function that fix the words problems

def transform_text(text):
    text = text.lower()
    
    r = re.findall('@[\w]*', text)
    
    for w in r:
        text = text.replace(w, ' ')
    text = text.replace('[^a-zA-Z#]', '')
    text = ' '.join([word for word in text.split() if len(word) > 2])
    
    text = re.sub(r'\$%,.\:\?', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    words = text.split(" ")
    words = [lemma.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    text = ' '.join(words) 
    
    return text

In [25]:
lemma = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\taha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
X = X.apply(lambda x: transform_text(x))


max_words = 4000
max_len = 150

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
tweets = pad_sequences(sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(tweets, y, test_size = 0.3)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [27]:
tokenizer.word_index

{'day': 1,
 'good': 2,
 'love': 3,
 'happy': 4,
 'get': 5,
 'like': 6,
 'work': 7,
 'time': 8,
 'going': 9,
 'today': 10,
 'one': 11,
 'lol': 12,
 'got': 13,
 'thanks': 14,
 'miss': 15,
 'really': 16,
 'know': 17,
 'back': 18,
 'want': 19,
 'great': 20,
 'fun': 21,
 'see': 22,
 'sad': 23,
 'sorry': 24,
 'home': 25,
 'well': 26,
 'hope': 27,
 'night': 28,
 'think': 29,
 'bad': 30,
 'new': 31,
 'feel': 32,
 'still': 33,
 'much': 34,
 'mother': 35,
 'need': 36,
 'wish': 37,
 'nice': 38,
 'make': 39,
 'hate': 40,
 'haha': 41,
 'morning': 42,
 'last': 43,
 'week': 44,
 'twitter': 45,
 'awesome': 46,
 'would': 47,
 'better': 48,
 'tomorrow': 49,
 'thing': 50,
 'look': 51,
 'right': 52,
 'come': 53,
 'suck': 54,
 'yeah': 55,
 'way': 56,
 'tonight': 57,
 'best': 58,
 'say': 59,
 'tired': 60,
 'though': 61,
 'thank': 62,
 'getting': 63,
 'dont': 64,
 'sick': 65,
 'gonna': 66,
 'sleep': 67,
 'friend': 68,
 'wait': 69,
 'people': 70,
 'cool': 71,
 'weekend': 72,
 'hurt': 73,
 'could': 74,
 'next'

In [29]:
embed_dim = 20
lstm_out = 15
max_words = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(max_words, embed_dim, input_length = max_len))
model.add(LSTM(lstm_out, dropout=0.4))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 10, batch_size = 128, validation_data = (X_test, y_test))
model.save('sentiment_analyzer.hdf5')

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 18s 94ms/step - accuracy: 0.5953 - loss: 0.9449 - val_accuracy: 0.7402 - val_loss: 0.7764
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - accuracy: 0.7483 - loss: 0.6828 - val_accuracy: 0.7827 - val_loss: 0.5858
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.8066 - loss: 0.5306 - val_accuracy: 0.8044 - val_loss: 0.5200
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.8342 - loss: 0.4616 - val_accuracy: 0.8038 - val_loss: 0.5078
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.8467 - loss: 0.4266 - val_accuracy: 0.8052 - val_loss: 0.5073
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - accuracy: 0.8583 - loss: 0.3990 - val_accuracy: 0.8023 - val_loss: 0.5123
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 75ms/step - accuracy: 0.8637 - loss: 0.3835 - val_accuracy: 0.8028 - val_loss: 0.5223
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 12s 81ms/step - accuracy: 0.8687 - loss: 0.3682 - 

In [30]:
y_pre = model.predict(X_test)
print(classification_report(np.argmax(y_test, axis = 1), np.argmax(y_pre, axis = 1)))

257/257 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
              precision    recall  f1-score   support

           0       0.83      0.78      0.80      3321
           1       0.72      0.80      0.76      2275
           2       0.83      0.82      0.82      2609

    accuracy                           0.80      8205
   macro avg       0.79      0.80      0.79      8205
weighted avg       0.80      0.80      0.80      8205



In [39]:
# Load trained model
model = tf.keras.models.load_model('sentiment_analyzer.hdf5')

def predict_sentiment(text):
    text = transform_text(text)
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=150)
    pred = model.predict(pad)[0]
    cls = np.argmax(pred)
    prob = max(pred)

    if cls == 0:
        return (f"😡 The sentiment is {prob} negative — sounds like frustration or disappointment.")
    elif cls == 1:
        return (f"😐 The sentiment is {prob} neutral — pretty balanced, no strong emotion.")
    else:
        return (f"😍 The sentiment is {prob} positive — lots of good vibes here!")


In [47]:
# Get user input
user_text = input("Enter a sentence: ")
print(predict_sentiment(user_text))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
😡 The sentiment is 0.9303151369094849 negative — sounds like frustration or disappointment.
